In [1]:
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import os, random
import torch
from transformers import BertTokenizer, BertForSequenceClassification

from CAP import CAPModel
from CAP.dataset import CAPDataSet
from CAP.utils import *

seed_everything(1813)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

## Functions

In [2]:
def preprocess_data(dataframe, tokenizer, max_len=64):
    input_ids = []
    attention_masks = []
    labels = []

    for _, row in dataframe.iterrows():
        encoded = tokenizer.encode_plus(
            str(row['texts'].upper()),  # 문자열로 변환하여 전달
            add_special_tokens=True,
            max_length=max_len,
            pad_to_max_length=True,
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True  # truncation 활성화
        )
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])
        labels.append(int(row['labels']))  # 정수형으로 변환하여 추가

    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    labels = torch.tensor(labels)  # 정수형 리스트를 텐서로 변환

    return TensorDataset(input_ids, attention_masks, labels)

## Define model

In [ ]:
model = BertForSequenceClassification.from_pretrained('bert-large-cased', num_labels=5).to(device)